this is just for practise

In [ ]:
from db_connection import get_connection
from tabulate import tabulate

creating db students 

In [ ]:
sql = """
DROP TABLE IF EXISTS source_grades;

-- Source data: Student grades
CREATE TABLE source_grades (
    student_email VARCHAR(100),
    student_name VARCHAR(100),
    course_code VARCHAR(20),
    course_name VARCHAR(100),
    instructor_name VARCHAR(100),
    grade_date DATE,
    score INT
);

INSERT INTO source_grades VALUES
('alice@school.edu', 'Alice', 'MATH101', 'Calculus I', 'Prof. Smith', '2024-01-15', 85),
('bob@school.edu', 'Bob', 'MATH101', 'Calculus I', 'Prof. Smith', '2024-01-15', 92),
('alice@school.edu', 'Alice', 'CS101', 'Programming', 'Prof. Jones', '2024-01-20', 88);

"""


In [ ]:

with get_connection() as connection:
    with connection.cursor() as cursor:
        cursor.execute(sql)
        connection.commit()   # REQUIRED for CREATE / INSERT



### YOUR TASK:
- 1. Create dim_students (student_key, student_email, student_name)
- 2. Create dim_courses (course_key, course_code, course_name, instructor_name)
- 3. Create dim_date (date_key, full_date, month_name, year)
- 4. Create fact_grades (grade_id, student_key, course_key, date_key, score)
- 5. Load dimensions with unique values
- 6. Load fact table by JOINing to get surrogate keys
- 7. Query: Average score per student
- 8. Query: Average score per course